# CUDA - GPU - ćwiczenia
**Paweł Kruczkiewicz**


TODO
  - pliki wrzucić tak jak trzeba, żeby się poprawnie kompilowało
  - podzielić logicznie cellki w notebooku
  - dodać odpowiednie kompilacje
  - napisać wnioski

Estimated time < 2h

## Przygotowanie środowiska wykonawczego

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-f0mhknr3
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-f0mhknr3
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=10d2b23ef240a00ca42dc12f6a7fe8f97890ffbebd3eab6e2218690e8956ed59
  Stored in directory: /tmp/pip-ephem-wheel-cache-j84d7k_t/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [5]:

!nvidia-smi


Tue Jun 13 14:15:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To Informatyka na WIET\n";
    return 0;
}

Welcome To Informatyka na WIET



In [30]:
%%cu
#include <cstdio>
#include <iostream>

	using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 0
The time required : 0.002688



## Przygotowanie środeowiska pod laby

In [7]:
!git clone https://github.com/pkrucz00/parallel-programming.git


Cloning into 'parallel-programming'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 346 (delta 74), reused 108 (delta 70), pack-reused 201
Receiving objects: 100% (346/346), 40.20 MiB | 21.56 MiB/s, done.
Resolving deltas: 100% (166/166), done.
fatal: not a git repository (or any of the parent directories): .git


In [8]:
%cd parallel-programming/
!git checkout gpu

/content/parallel-programming
Branch 'gpu' set up to track remote branch 'gpu' from 'origin'.
Switched to a new branch 'gpu'


In [9]:


!nvidia-smi
!free -g

Tue Jun 13 14:19:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [72]:
%cd gpu/CUDA-Lab02

[Errno 2] No such file or directory: 'gpu/CUDA-Lab02'
/content/parallel-programming/gpu/CUDA-Lab02


In [70]:
!ls

1  2  CUDA-Lab02  data	sprawozdanie-3.pdf  sprawozdanie.ipynb


In [14]:
## Ćwiczenie 1

### Reduction

In [73]:
%cd "1 Reduction"

/content/parallel-programming/gpu/CUDA-Lab02/1 Reduction


In [16]:
!ls

analyser.bat	reduction_global.cpp	    reduction_shared.cpp
exception.h	reduction_global_gpu.bat    reduction_shared_kernel.cu
helper_timer.h	reduction_global_kernel.cu
profiler.bat	reduction.h


**Globalna**

In [33]:
!nvcc -o reduction_global_gpu reduction_global.cpp reduction_global_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.


In [36]:
!chmod 777 reduction_global_gpu
!./reduction_global_gpu

Time= 14.468 msec, bandwidth= 4.638505 GB/s
host: 0.996007, device 0.996007


In [78]:
!nvprof ./reduction_global_gpu

==24861== NVPROF is profiling process 24861, command: ./reduction_global_gpu
Time= 14.640 msec, bandwidth= 4.584061 GB/s
host: 0.996007, device 0.996007
==24861== Profiling application: ./reduction_global_gpu
==24861== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   95.23%  1.40321s      2424  578.88us  451.42us  790.87us  global_reduction_kernel(float*, float*, int, int)
                    3.74%  55.136ms       100  551.36us  543.51us  557.59us  [CUDA memcpy DtoD]
                    1.02%  15.077ms         1  15.077ms  15.077ms  15.077ms  [CUDA memcpy HtoD]
                    0.00%  1.7600us         1  1.7600us  1.7600us  1.7600us  [CUDA memcpy DtoH]
      API calls:   81.86%  1.44307s       101  14.288ms  6.3930us  34.923ms  cudaDeviceSynchronize
                   16.06%  283.21ms         2  141.61ms  227.16us  282.99ms  cudaMalloc
                    1.04%  18.368ms       102  180.08us  20.505us  15.284ms  cu

In [75]:
!nvprof --analysis-metrics -o reduction_global_gpu.nvvp -f  ./reduction_global_gpu

======== Warning: Skipping profiling on device 0 since profiling is not supported on devices with compute capability 7.5 and higher.
                  Use NVIDIA Nsight Compute for GPU profiling and NVIDIA Nsight Systems for GPU tracing and CPU sampling.
                  Refer https://developer.nvidia.com/tools-overview for more details.

======== Warning: The option --aggregate-mode on has no effect. The --aggregate-mode <on|off> option applies to --events and --metrics options that follow it.
======== Warning: The option --aggregate-mode off has no effect. The --aggregate-mode <on|off> option applies to --events and --metrics options that follow it.
==23995== NVPROF is profiling process 23995, command: ./reduction_global_gpu
Time= 14.424 msec, bandwidth= 4.652674 GB/s
host: 0.996007, device 0.996007
==23995== Generated result file: /content/parallel-programming/gpu/CUDA-Lab02/1 Reduction/reduction_global_gpu.nvvp


/bin/bash: nvvp: command not found


In [67]:
%cd ..

/content/parallel-programming/gpu


In [50]:
## Ćwiczenie 2

In [51]:
%cd "2 Warp Divergence"

/content/parallel-programming/gpu/CUDA-Lab02/2 Warp Divergence


In [62]:
!nvcc -o reduction_interleaving reduction.cpp reduction_kernel_interleaving.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [63]:
!nvcc -o reduction_sequential reduction.cpp reduction_kernel_sequential.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [64]:
!chmod 744 reduction_interleaving
!chmod 744 reduction_sequential

!./reduction_interleaving
!./reduction_sequential

Time= 3.025 msec, bandwidth= 22.186216 GB/s
host: 0.996007, device 0.996007
Time= 2.490 msec, bandwidth= 26.953516 GB/s
host: 0.996007, device 0.996007


In [65]:
%cd ..

/content/parallel-programming/gpu/CUDA-Lab02


In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>

#define BLOCK_SIZE 32

__global__ void matrix_transpose_naive(int *input, int *output, int n) {

	int indexX = threadIdx.x + blockIdx.x * blockDim.x;
	int indexY = threadIdx.y + blockIdx.y * blockDim.y;
	int index = indexY * n + indexX;
	int transposedIndex = indexX * n + indexY;

    // this has discoalesced global memory store
	output[transposedIndex] = input[index];

	// this has discoalesced global memore load
	// output[index] = input[transposedIndex];
}

__global__ void matrix_transpose_shared(int *input, int *output, int n) {

	__shared__ int sharedMemory [BLOCK_SIZE] [BLOCK_SIZE];

	// global index
	int indexX = threadIdx.x + blockIdx.x * blockDim.x;
	int indexY = threadIdx.y + blockIdx.y * blockDim.y;

	// transposed global memory index
	int tindexX = threadIdx.x + blockIdx.y * blockDim.x;
	int tindexY = threadIdx.y + blockIdx.x * blockDim.y;

	// local index
	int localIndexX = threadIdx.x;
	int localIndexY = threadIdx.y;

	int index = indexY * n + indexX;
	int transposedIndex = tindexY * n + tindexX;

	// reading from global memory in coalesed manner and performing tanspose in shared memory
	sharedMemory[localIndexX][localIndexY] = input[index];

	__syncthreads();

	// writing into global memory in coalesed fashion via transposed data in shared memory
	output[transposedIndex] = sharedMemory[localIndexY][localIndexX];
}

//basically just fills the array with index.
void fill_array(int *data, int n) {
	for(int idx=0;idx<(n*n);idx++)
		data[idx] = idx;
}

void print_output(int *a, int *b, int n) {
	printf("\n Original Matrix::\n");
	for(int idx=0;idx<(n*n);idx++) {
		if(idx%n == 0)
			printf("\n");
		printf(" %d ",  a[idx]);
	}
	printf("\n Transposed Matrix::\n");
	for(int idx=0;idx<(n*n);idx++) {
		if(idx%n == 0)
			printf("\n");
		printf(" %d ",  b[idx]);
	}
}

int main(void) {
	int *a, *b;
        int *d_a, *d_b; // device copies of a, b, c

	cudaEvent_t start, end;
  cudaEventCreate(&start);
  cudaEventCreate(&end);

  for (int n = 1028; n <= 10280; n += 1028){
    int size = n * n *sizeof(int);

    // Alloc space for host copies of a, b, c and setup input values
    a = (int *)malloc(size); fill_array(a, n);
    b = (int *)malloc(size);

    // Alloc space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);

    // Copy inputs to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 blockSize(BLOCK_SIZE,BLOCK_SIZE,1);
    dim3 gridSize(n/BLOCK_SIZE,n/BLOCK_SIZE,1);

    for (int i = 0; i < 5; i++){
    cudaEventRecord(start);
    // matrix_transpose_naive<<<gridSize,blockSize>>>(d_a,d_b,n);
    matrix_transpose_shared<<<gridSize,blockSize>>>(d_a,d_b,n);
    cudaEventRecord(end);
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);
    printf("%d,%f,%d\n", n, time, i);
}
    free(a);
    free(b);
      cudaFree(d_a);
    cudaFree(d_b);
  }

	return 0;
}

1028,0.141408,0
1028,0.145376,1
1028,0.139424,2
1028,0.139296,3
1028,0.139264,4
2056,0.536928,0
2056,0.528640,1
2056,0.525984,2
2056,0.528352,3
2056,0.527360,4
3084,1.195296,0
3084,1.194752,1
3084,1.190944,2
3084,1.189920,3
3084,1.193536,4
4112,2.079040,0
4112,2.077408,1
4112,2.069216,2
4112,2.078080,3
4112,2.073216,4
5140,3.305408,0
5140,3.300992,1
5140,3.303456,2
5140,3.301920,3
5140,3.300384,4
6168,4.689760,0
6168,4.685184,1
6168,4.690880,2
6168,4.700512,3
6168,4.691936,4
7196,6.504000,0
7196,6.507616,1
7196,6.503136,2
7196,6.498848,3
7196,6.500480,4
8224,8.243040,0
8224,8.241408,1
8224,8.251936,2
8224,8.252288,3
8224,8.240960,4
9252,10.611904,0
9252,10.605984,1
9252,10.608608,2
9252,10.605952,3
9252,10.600768,4
10280,11.835904,0
10280,11.829248,1
10280,11.840032,2
10280,11.839360,3
10280,11.839456,4

